This notebook is made for detecting arena edges and cropping videos accordingly. It is focused on 6 corridors pushing arenas and has not been adapted to crop top and bottom  of the arena.

In [ ]:

import cv2
import pathlib
import numpy as np
import matplotlib.pyplot as plt
%matplotlib widget
import seaborn as sns
import more_itertools as mit
from pathlib import Path
import os
from scipy import signal
import holoviews as hv
from holoviews import opts
hv.extension('bokeh',
             #'matplotlib',
             )

import sys

sys.path.insert(0, "..")

from Utilities.Utils import *
from Utilities.Processing import *

import black
import jupyter_black

jupyter_black.load()

# Load video path and get first frame

In [ ]:
VideoPath = pathlib.Path(
    "/mnt/labserver/DURRIEU_Matthias/Experimental_data/Optogenetics/Optobot/MultiMaze_15stepped_gated_bowtie/Starved_noWater/230209/111026_s0a0_p0-0/MultiMaze_15stepped_gated_bowtie_Starved_noWater_p0-0_80fps.mp4"
)
vidcap = cv2.VideoCapture(VideoPath.as_posix())
for i in range(1):
    success, im_full = vidcap.read()

im_full_gray = cv2.cvtColor(im_full, cv2.COLOR_BGR2GRAY)

plt.figure()
plt.imshow(im_full_gray, cmap="gray", vmin=0, vmax=255)

# Display gray values profile

In [ ]:
cols = im_full_gray.sum(axis=0)

hv.Histogram(cols).opts(tools=["hover"])

# Detect peaks

In [ ]:
peaks = signal.find_peaks(
    cols,
    distance=40,
    height=35_000,
)

# Check that peaks are correctly located

x = np.array(range(0, len(cols)))
PeaksPos = (x[peaks[0]], cols[peaks[0]])
hv.Histogram(cols).opts(tools=["hover"]) * hv.Points(PeaksPos).opts(
    color="orange", tools=["hover"]
)

In [ ]:
print(len(peaks[0]))

# Generate arenas zones coordinates and add tails

In [ ]:
%matplotlib inline

ArenaList = []
for i in range(0, len(peaks[0])):
    if (i % 2) == 0:
        ArenaList.append(list(range(peaks[0][i] - 40, peaks[0][i + 1] + 40)))

# Visual test if crop was successful
plt.imshow(im_full_gray[:, ArenaList[5]], cmap="gray", vmin=0, vmax=255)

# Make folder in which each cropped video will be stored

In [ ]:
for n in range(1, 7):
    os.mkdir(VideoPath.parent.joinpath("Arena" + str(n)))

# Build videowriters

In [ ]:
cap = cv2.VideoCapture(VideoPath.as_posix())

# Writer parameters
codec = "mp4v"
fourcc = cv2.VideoWriter_fourcc(*codec)
A1 = cv2.VideoWriter(
    filename=VideoPath.parent.joinpath("Arena1/Arena1.mp4").as_posix(),
    fourcc=fourcc,
    fps=80.0,
    frameSize=(len(ArenaList[0]), int(cap.read()[1].shape[0])),
    isColor=True,
)
A2 = cv2.VideoWriter(
    filename=VideoPath.parent.joinpath("Arena2/Arena2.mp4").as_posix(),
    fourcc=fourcc,
    fps=80.0,
    frameSize=(len(ArenaList[1]), int(cap.read()[1].shape[0])),
    isColor=True,
)

A3 = cv2.VideoWriter(
    filename=VideoPath.parent.joinpath("Arena3/Arena3.mp4").as_posix(),
    fourcc=fourcc,
    fps=80.0,
    frameSize=(len(ArenaList[2]), int(cap.read()[1].shape[0])),
    isColor=True,
)

A4 = cv2.VideoWriter(
    filename=VideoPath.parent.joinpath("Arena4/Arena4.mp4").as_posix(),
    fourcc=fourcc,
    fps=80.0,
    frameSize=(len(ArenaList[3]), int(cap.read()[1].shape[0])),
    isColor=True,
)

A5 = cv2.VideoWriter(
    filename=VideoPath.parent.joinpath("Arena5/Arena5.mp4").as_posix(),
    fourcc=fourcc,
    fps=80.0,
    frameSize=(len(ArenaList[4]), int(cap.read()[1].shape[0])),
    isColor=True,
)

A6 = cv2.VideoWriter(
    filename=VideoPath.parent.joinpath("Arena6/Arena6.mp4").as_posix(),
    fourcc=fourcc,
    fps=80.0,
    frameSize=(len(ArenaList[5]), int(cap.read()[1].shape[0])),
    isColor=True,
)

scaling = 1.0

# Write cropped videos

In [ ]:
last = 0

while True:
    ret, frame = cap.read()  # Grab frame
    this = cap.get(1)
    if ret == True:

        # frame = cv2.resize(frame, None, fx=scaling, fy=scaling,
        # interpolation=cv2.INTER_LINEAR)

        ImGr = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # convert to grayscale
        # (height, width) = frame.shape[:2]

        Arena1 = frame[:, ArenaList[0]]
        Arena2 = frame[:, ArenaList[1]]
        Arena3 = frame[:, ArenaList[2]]
        Arena4 = frame[:, ArenaList[3]]
        Arena5 = frame[:, ArenaList[4]]
        Arena6 = frame[:, ArenaList[5]]

        # Display each arena in a dedicated window (here it will be replaced by cv2 video writer in the end
        # cv2.imshow('Arena1', Arena1)
        # cv2.imshow('Arena2', Arena2)
        # cv2.imshow('Arena3', Arena3)
        # cv2.imshow('Arena4', Arena4)
        # cv2.imshow('Arena5', Arena5)
        # cv2.imshow('Arena6', Arena6)

        A1.write(Arena1)
        A2.write(Arena2)
        A3.write(Arena3)
        A4.write(Arena4)
        A5.write(Arena5)
        A6.write(Arena6)

        if cv2.waitKey(1) == 27:
            exit(0)
    if last >= this:
        break
    last = this
cap.release()
A1.release()
A2.release()
A3.release()
A4.release()
A5.release()
A6.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

> Code speed is way higher if cropped videos are not displays during the process. If you want to display them, uncomment the "cv2.imshow" lines.

# Crop an horizontal layout arena

In [ ]:
VideoPath = pathlib.Path(
    "/mnt/labserver/DURRIEU_Matthias/Code/Sleap_Pretrained_Maxime/TrainingVideo/multiMazeTrimmed1.mp4"
)
vidcap = cv2.VideoCapture(VideoPath.as_posix())
for i in range(1):
    success, im_full = vidcap.read()

im_full_gray = cv2.cvtColor(im_full, cv2.COLOR_BGR2GRAY)

plt.figure()
plt.imshow(im_full_gray, cmap="gray", vmin=0, vmax=255)

In [ ]:
rows = im_full_gray.sum(axis=1)

hv.Histogram(rows).opts(tools=["hover"])

In [ ]:
peaks = signal.find_peaks(
    rows,
    distance=40,
    height=30_000,
)

# Check that peaks are correctly located

x = np.array(range(0, len(rows)))
PeaksPos = (x[peaks[0]], rows[peaks[0]])
hv.Histogram(rows).opts(tools=["hover"]) * hv.Points(PeaksPos).opts(
    color="orange", tools=["hover"]
)

> Interestingly when looking at x values instead of y, the information on frame, timing and all displayed on top left produces a sharp peak that needs to be removed before cropping.

In [ ]:
peaklocs = peaks[0][1:]

In [ ]:
%matplotlib inline

ArenaList = []
for i in range(0, len(peaklocs)):
    if (i % 2) == 0:
        ArenaList.append(list(range(peaklocs[i] - 40, peaklocs[i + 1] + 40)))

# Visual test if crop was successful
plt.imshow(im_full_gray[ArenaList[5], :], cmap="gray", vmin=0, vmax=255)

In [ ]:
ArenaList[0]

# Save cropping parameters for later use

In [ ]:
# Get the first and last value of each element in ArenaList
CroppingParams = []
for i in range(0, len(ArenaList)):
    CroppingParams.append([ArenaList[i][0], ArenaList[i][-1]])

CroppingParams

In [ ]:
CroppingParams[0]

Save cropping parameters for later use

In [ ]:
checksave(
    path=VideoPath.parent.joinpath("Arena_indices.npy"),
    object="parameter",
    file=CroppingParams,
)

In [ ]:
for n in range(1, 7):
    os.mkdir(VideoPath.parent.joinpath("Arena" + str(n)))

In [ ]:
cap = cv2.VideoCapture(VideoPath.as_posix())

fps = 40.0

# Writer parameters
codec = "mp4v"
fourcc = cv2.VideoWriter_fourcc(*codec)
A1 = cv2.VideoWriter(
    filename=VideoPath.parent.joinpath("Arena1/Arena1.mp4").as_posix(),
    fourcc=fourcc,
    fps=fps,
    frameSize=(int(cap.read()[1].shape[0]), len(ArenaList[0])),
    isColor=True,
)
A2 = cv2.VideoWriter(
    filename=VideoPath.parent.joinpath("Arena2/Arena2.mp4").as_posix(),
    fourcc=fourcc,
    fps=fps,
    frameSize=(int(cap.read()[1].shape[0]), len(ArenaList[1])),
    isColor=True,
)

A3 = cv2.VideoWriter(
    filename=VideoPath.parent.joinpath("Arena3/Arena3.mp4").as_posix(),
    fourcc=fourcc,
    fps=fps,
    frameSize=(int(cap.read()[1].shape[0]), len(ArenaList[2])),
    isColor=True,
)

A4 = cv2.VideoWriter(
    filename=VideoPath.parent.joinpath("Arena4/Arena4.mp4").as_posix(),
    fourcc=fourcc,
    fps=fps,
    frameSize=(int(cap.read()[1].shape[0]), len(ArenaList[3])),
    isColor=True,
)

A5 = cv2.VideoWriter(
    filename=VideoPath.parent.joinpath("Arena5/Arena5.mp4").as_posix(),
    fourcc=fourcc,
    fps=fps,
    frameSize=(int(cap.read()[1].shape[0]), len(ArenaList[4])),
    isColor=True,
)

A6 = cv2.VideoWriter(
    filename=VideoPath.parent.joinpath("Arena6/Arena6.mp4").as_posix(),
    fourcc=fourcc,
    fps=fps,
    frameSize=(int(cap.read()[1].shape[0]), len(ArenaList[5])),
    isColor=True,
)

scaling = 1.0

In [ ]:
last = 0

while True:
    ret, frame = cap.read()  # Grab frame
    this = cap.get(1)
    if ret == True:

        # frame = cv2.resize(frame, None, fx=scaling, fy=scaling,
        # interpolation=cv2.INTER_LINEAR)

        ImGr = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # convert to grayscale
        # (height, width) = frame.shape[:2]

        Arena1 = frame[ArenaList[0], :]
        Arena2 = frame[ArenaList[1], :]
        Arena3 = frame[ArenaList[2], :]
        Arena4 = frame[ArenaList[3], :]
        Arena5 = frame[ArenaList[4], :]
        Arena6 = frame[ArenaList[5], :]

        # Display each arena in a dedicated window (here it will be replaced by cv2 video writer in the end
        # cv2.imshow('Arena1', Arena1)
        # cv2.imshow('Arena2', Arena2)
        # cv2.imshow('Arena3', Arena3)
        # cv2.imshow('Arena4', Arena4)
        # cv2.imshow('Arena5', Arena5)
        # cv2.imshow('Arena6', Arena6)

        A1.write(Arena1)
        A2.write(Arena2)
        A3.write(Arena3)
        A4.write(Arena4)
        A5.write(Arena5)
        A6.write(Arena6)

        # Display a progress bar
        sys.stdout.write("Frame: " + str(this) + " / " + str(cap.get(7)) + "\r")
        # sys.stdout.write("Frame: " + str(this) + " / " + str(cap.get(7)) + " - " + str(round(this/cap.get(7)*100,2)) + "%\r")
        sys.stdout.flush()

        if cv2.waitKey(1) == 27:
            exit(0)
    if last >= this:
        break
    last = this
cap.release()
A1.release()
A2.release()
A3.release()
A4.release()
A5.release()
A6.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

Check frame accuracy

In [ ]:
# TODO: Check progress accuracy by comparing key frames between original and cropped videos